## Подготовка
Импортируем numpy и загрузим данные:

In [2]:
import numpy as np

marks = np.loadtxt("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/data_hw1.txt")

## Число студентов
Посчитаем количество студентов:

In [3]:
num_studs = marks.shape[0]
print(num_studs)

28


## Очень сложная проверочная
Найдём максимальный балл за первую проверочную, поделим весь первый столбец на него и умножим на 10:

In [4]:
max_mark = np.max(marks[:, 0])
print("Max mark = ", max_mark)

print(marks[:, 0])
marks[:, 0] = marks[:, 0] / max_mark * 10
print(marks[:, 0])

Max mark =  7.0
[7.  0.  3.  3.4 6.8 7.  6.  0.  5.  0.  3.  4.  0.  6.  5.  0.  7.  4.5
 0.  0.  0.  6.  6.5 7.  0.  7.  3.  3.5]
[10.          0.          4.28571429  4.85714286  9.71428571 10.
  8.57142857  0.          7.14285714  0.          4.28571429  5.71428571
  0.          8.57142857  7.14285714  0.         10.          6.42857143
  0.          0.          0.          8.57142857  9.28571429 10.
  0.         10.          4.28571429  5.        ]


## Подсчёт накопительной оценки
Посчитаем накоп для каждого студента: подсчитаем средние по проверочным, домашним работам и коллоквиуму, умножим на веса и сложим

In [5]:
# static
weights = [0.3, 0.4, 0.3]
assignment_columns = [0, 1, 2]
home_assignment_columns = [3, 4, 5]
colloquium_column = [6]
all_rows = range(num_studs)

# extract specific columns to distinct arrays
assignments_array = marks[np.ix_(all_rows, assignment_columns)]
home_assignments_array = marks[np.ix_(all_rows, home_assignment_columns)]
colloquium_array = marks[np.ix_(all_rows, colloquium_column)]

# calculate means
assignment_means = np.mean(assignments_array, axis=1)
home_assignment_means = np.mean(home_assignments_array, axis=1)
colloquium_mean = colloquium_array

# reshape to N x 1 matrix to merge with original matrix with N rows
assignment_means = np.reshape(assignment_means, (num_studs, 1))
home_assignment_means = np.reshape(home_assignment_means, (num_studs, 1))

# calculate new column
new_column = np.zeros((num_studs, 1))
new_column = new_column + 0.3 * assignment_means + 0.4 * home_assignment_means + 0.3 * colloquium_mean

marks = np.hstack((marks, new_column))

print(new_column)
print(marks)

[[3.53      ]
 [3.53      ]
 [8.23857143]
 [5.72238095]
 [6.1247619 ]
 [9.2       ]
 [8.96380952]
 [1.5       ]
 [9.46761905]
 [8.56333333]
 [1.90857143]
 [9.21809524]
 [0.        ]
 [8.71714286]
 [8.63428571]
 [1.34      ]
 [9.96      ]
 [3.77952381]
 [0.        ]
 [1.43      ]
 [0.        ]
 [8.51714286]
 [9.78857143]
 [9.56666667]
 [3.04666667]
 [4.84      ]
 [9.01190476]
 [8.81666667]]
[[10.          8.         10.          0.          0.          2.1
   1.5         3.53      ]
 [ 0.          6.          4.4         4.          0.          5.
   4.3         3.53      ]
 [ 4.28571429  7.          8.7         7.6         7.          9.7
  10.          8.23857143]
 [ 4.85714286  1.3         7.          3.4         0.          9.4
   9.          5.72238095]
 [ 9.71428571  9.1         8.2         0.          0.          7.9
   7.9         6.1247619 ]
 [10.         10.         10.          9.          9.5        10.
   8.          9.2       ]
 [ 8.57142857 10.          9.7         8.7   

## Студенты, которых, вероятно, отчислят
Подсчитаем количество студентов с накопительной оценкой, равной 0:

In [6]:
column_of_zeros = np.zeros((num_studs, 1))
result_of_comparing = (column_of_zeros == new_column)
num_of_students_for_expulsion = np.sum(result_of_comparing)
print(num_of_students_for_expulsion)

3


## Студенты, которые получат автомат
Подсчитаем количество студентов, которые получат автомат - их накоп должен быть больше 7. Присоединим к матрице новую колонку, показывающую, автомат ли у студента

In [7]:
autopass_column = np.array(new_column > 7).astype(int)
marks = np.hstack((marks, autopass_column))
print(marks)

[[10.          8.         10.          0.          0.          2.1
   1.5         3.53        0.        ]
 [ 0.          6.          4.4         4.          0.          5.
   4.3         3.53        0.        ]
 [ 4.28571429  7.          8.7         7.6         7.          9.7
  10.          8.23857143  1.        ]
 [ 4.85714286  1.3         7.          3.4         0.          9.4
   9.          5.72238095  0.        ]
 [ 9.71428571  9.1         8.2         0.          0.          7.9
   7.9         6.1247619   0.        ]
 [10.         10.         10.          9.          9.5        10.
   8.          9.2         1.        ]
 [ 8.57142857 10.          9.7         8.7         7.          8.5
   9.7         8.96380952  1.        ]
 [ 0.          9.3         5.7         0.          0.          0.
   0.          1.5         0.        ]
 [ 7.14285714 10.         10.          9.9        10.5        10.
   9.          9.46761905  1.        ]
 [ 0.          8.7         8.          9.7        

## Студенты, освобождённые от задач на экзамене
Отсортируем студентов по накопительной оценке и возьмём первую половину - они будут освобождены от задач на экзамене

In [8]:
cumulative_column = 7
sorted_marks = marks[marks[:, cumulative_column].argsort()][::-1]
free_from_practicum_students = sorted_marks[np.ix_(range(num_studs // 2))]
print(free_from_practicum_students)

[[10.         10.         10.          9.7        10.         10.
  10.          9.96        1.        ]
 [ 9.28571429  8.7         9.5        10.5         9.8        10.
  10.          9.78857143  1.        ]
 [10.          9.7        10.          8.1        10.         10.
   9.5         9.56666667  1.        ]
 [ 7.14285714 10.         10.          9.9        10.5        10.
   9.          9.46761905  1.        ]
 [ 5.71428571 10.         10.          9.9         9.7        10.
   9.          9.21809524  1.        ]
 [10.         10.         10.          9.          9.5        10.
   8.          9.2         1.        ]
 [ 4.28571429  8.7        10.          9.4        10.7        10.
   9.          9.01190476  1.        ]
 [ 8.57142857 10.          9.7         8.7         7.          8.5
   9.7         8.96380952  1.        ]
 [ 5.          9.3        10.          9.9        10.         10.
   8.          8.81666667  1.        ]
 [ 8.57142857  5.         10.          9.7        10. 

## Сложность заданий
Определим весы для каждого студента и сосчитаем сложность каждого задания

In [20]:
weights_for_complexity = np.array([1.]*7+[2.]*4+[3.]*7+[4.]*(marks.shape[0]-18))
weights_for_complexity = weights_for_complexity / np.sum(weights_for_complexity)
lower_borders = [0, 7, 11, 18]
lengths = [7, 4, 7, 10]

complexities = []

for i in range(7):
    complexity = 0
    for j in range(4):
        start = lower_borders[j]
        end = lower_borders [j] + lengths[j]
        complexity += np.sum(marks[np.ix_(range(start, end), [i])].flatten() * weights_for_complexity[np.ix_(range(start, end))])
    complexities.append(complexity)

print(complexities)

[4.900375939849624, 7.1197368421052625, 7.8578947368421055, 6.164473684210526, 4.832894736842105, 5.878947368421052, 4.873684210526315]
